In [ ]:
# Libraries for downloading data from FTP
import shutil
import urllib.request as req
from contextlib import closing

# Library for uploading data to S3
import boto3

# Libraries for handling data
import rasterio as rio
import pandas as pd
import numpy as np

# Libraries for various helper functions
from datetime import datetime
import os
import threading
import sys

cli.005.1 https://nsidc.org/data/docs/noaa/g02135_seaice_index/#text_summary
SRC: ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/01_Jan (etc)
files look like: N_197901_concentration_v2.1.tif, N_197901_extent_v2.1.tif
from Jan 1979
File type: tif
North Polar Projection

cli.005.2 https://nsidc.org/data/docs/noaa/g02135_seaice_index/#text_summary
SRC: ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/01_Jan (etc)
files look like: S_197901_concentration_v2.1.tif
from Jan 1979
File type: tif
Antarctic Projection

In [ ]:
## Need to know - how often do they update?

# This could run every day... there won't necessarily be anything to add
# In that case perhaps it should return an error to some console
# It would be nice for RW staff to see what has been updated
# And what updates were attempted but found nothing new


In [ ]:
# Directions for how to read contents of an FTP directory
# https://stackoverflow.com/questions/2289768/python-and-urllib
remote_path_north = "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/north/monthly/geotiff/"
remote_path_south = "ftp://sidads.colorado.edu/DATASETS/NOAA/G02135/south/monthly/geotiff/"
file = req.urlopen(remote_path_north).read().splitlines()
file

In [ ]:
def format_month(mon):
    if mon < 10:
        return("0" + str(mon))
    else:
        return(str(mon))

now = datetime.now()
year = str(now.year)
month = format_month(now.month-1)

def create_most_recent_file(year, month, north_or_south="N"):
    # file_type in [extent, concentration]
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
              "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    return("{}_{}/{}_{}{}_extent_v2.1.tif".format(month,months[int(month)-1],north_or_south, year, month))

arctic_file = create_most_recent_file(year, month, "N")
antarctic_file = create_most_recent_file(year, month, "S")

In [ ]:
antarctic_file

In [ ]:
ftp_arctic = remote_path_north+arctic_file
ftp_antarctic = remote_path_south+antarctic_file
print(ftp_arctic)
print(ftp_antarctic)

local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/Polar_Sea_Ice/"
# Use 7: to not include the month's folder in the local file name
arctic_file_name = arctic_file[7:]
antarctic_file_name = antarctic_file[7:]

local_arctic_orig = local_folder+arctic_file_name
local_antarctic_orig = local_folder+antarctic_file_name

local_arctic_edit = local_arctic[:-4] + "_edit.tif"
local_antarctic_edit = local_antarctic[:-4] + "_edit.tif"

with closing(req.urlopen(ftp_arctic)) as r:
    with open(local_arctic, 'wb') as f:
        shutil.copyfileobj(r, f)
         
with closing(req.urlopen(ftp_antarctic)) as r:
    with open(local_antarctic, 'wb') as f:
        shutil.copyfileobj(r, f)

In [ ]:
def reproject_file(orig_name, edit_name):
    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example
        
        srcprofile = src.profile
        rows = srcprofile["height"]
        columns = srcprofile["width"]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = srcprofile["nodata"]
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }

        windows = src.block_windows()
        with rio.open(edit_name, "w", **profile) as dst:
            for ix, window in windows:
                data = src.read(1, window=window)
                dst.write(data.astype(profile["dtype"]), 1, window=window)

reproject_file(local_arctic_orig, local_arctic_edit)
reproject_file(local_antarctic_orig, local_antarctic_edit)

In [ ]:
with rio.open(local_arctic_edit) as src:
    pro = src.profile
    print(pro)
    
with rio.open(local_antarctic_edit) as src:
    pro = src.profile
    print(pro)

In [ ]:
### NOTE = THIS WILL KEEP REPLACING THE FILE WITH THE NEWEST VERSION

# S3 destinations
s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/"

s3_file_antarctic = "cli_005_antarctic_sea_ice_extent.tif"
s3_key_antarctic_orig = s3_folder + s3_file_antarctic
s3_key_antarctic_edit = s3_key_antarctic_orig[:-4] + "_edit.tif"

s3_file_arctic = "cli_006_arctic_sea_ice_extent.tif"
s3_key_arctic_orig = s3_folder + s3_file_arctic
s3_key_arctic_edit = s3_key_arctic_orig[:-4] + "_edit.tif"

# S3 services
s3_download = boto3.resource("s3")
s3_upload = boto3.client("s3")

# Helper function to view upload progress
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [ ]:
# Originals
s3_upload.upload_file(local_arctic, s3_bucket, s3_key_arctic_orig,
                         Callback=ProgressPercentage(local_arctic))

s3_upload.upload_file(local_antarctic, s3_bucket, s3_key_antarctic_orig,
                         Callback=ProgressPercentage(local_antarctic))

# Edits
s3_upload.upload_file(local_arctic_edit, s3_bucket, s3_key_arctic_edit,
                         Callback=ProgressPercentage(local_arctic_edit))

s3_upload.upload_file(local_antarctic_edit, s3_bucket, s3_key_antarctic_edit,
                         Callback=ProgressPercentage(local_antarctic_edit))